## 서비스 (Servics)

### 1. 서비스란?
- 외부에서 클러스터 내의 **여러개의 Pod에 접근할 수 있는 하나의 IP를 제공**
- 근본적으로는 **로드 밸런서** 역할을 하며 다양한 서비스 제공
- 클러스터 내에서 노드를 옮겨 다니며 **동적으로 변하는 파드들에 고정적인 접근**을 위한 방법
- 파드가 클러스터 안 다른 위치로 옮겨져 IP변하더라도 자동으로 위치가 변한 파드와 통신
- **라벨 셀럭터리를 통한 특정 라벨을 가지고 있는 Pod선택하여 서비스**로 묶음
- 파드의 ip가 영속적이지 않아 항상 변할 수 있음
- **여러 개의 디플로이먼트를 하나의 완벽한 어플리케이션으로 연동하려면 파드 IP가 아닌 서로를 발견할 수 있는 방법이 필요**
- 일반적으로 Deployment 에서는 내부 포트만 정의, 외부에서 이 포드에 접근하기 위해선 서비스를 생성해야 함.
<hr>



### 2. 서비스 타입
- (1) ClusterIP
    - 기본 서비스 타입으로 클러스터 IP를 이용해서 서비스 연결 파드 접근
    - **쿠버네티스 내부에서만 파드들에 접근할 때 사용**
열어두는 것, 포드들에 접근시 내부적으로 사용하는 포트**
- (2) NodePort
    - 서비스 하나에 모든 노드의 지정된 포트 할당 
    - ex) node1:8080, node2:8080 -> 노드에 상관없이 지정된 포트로 파드 접근 가능
    - **외부에서 파드에 접근할 수 있는 서비스 타입**
    - 사실 NodePort로 서비스를 외부에 제공하는 경우는 거의 X
        - SSL 인증서 적용, 라우팅 등과 같은 설정을 서비스에 적용하기가 어렵기 때문
    - 일반적으로는 NodePort 서비스를 외부로 제공하기 보다는 인그레스(Ingress)라는 쿠버네틱스의 오브젝트로 사용되는 경우가 일반적
- (3) LoadBalancer
    - aws, gcp와 같은 Public cloud service, private cloud 용 로드 밸런서 이용 방법
    - **외부에서 파드에 접근할 수 있는 서비스 타입**
    - **클라우드에서 제공하는 로드 밸런서를 동적으로 프로비저닝해 포드에 연결**
- (4) ExternalName
    - 클러스터안에서 외부에 접근할 대 주로 사용
    - 서비스를 spec.externalNane필드에 설정한 값과 연결하여 사용


![kubernetes_sevice](img/kubernetes_sevice.PNG)
- 생성된 서비스는 쿠버네티스 내부에서만 사용할 수 있는 고유한 IP(ClusterIP)를 할당 받음.
- **Port** : 해당 서비스의 ip로 접근할 때 사용하는 포트(열려 있는 포트)라는 뜻
- **TargetPort**(== 포드의 containerPort 항목) : **해당 서비스의 Selector 라벨들을 통해 접근 대상이 된 파드들에 대해 동일한 포트로 접근**

- NodePort 설명
    - nodeport는 cluster-ip 10.110.14.145와 연결되어 있고 node에 접속하여, 30080으로 요청 가능 
    - 클러스터의 모든 노드에 내부 IP또는 외부 IP를 통해 30080으로 접근시 동일한 서비스에 연결 가능
   

<hr>

### 3. Kube-proxy
- 서비스를 만들시, 클러스터 IP나 노드 포트로 접근할 수 있게 실제 조작을 하는 컴포넌트
- 쿠버네티스의 노드마다 실행되어 클러스터 내부 ip로 연결하려는 요청을 적절한 파드로 전달
- Kube-proxy가 네트워크를 관리하는 모드 3가지 userspace, iptables, IPVS 방법 존재
- 기본 userspace(왼쪽) -> 현재 iptables(오른쪽) -> 나중 IPVS

![kubernetes_kube_proxy](img/kubernetes_kube_proxy.png)

- (1) userspace
    - 서비스 클러스터 IP요청 -> iptables 거쳐 kube-proxy 전달 -> 서비스 cluter ip가 적절한 파드연결
- (2) iptables 모드 
    - kube-proxy가 iptables만 관리, 클라이언트 IP 요청 -> iptables이 받아 적절한 파드에 직접 전달
    - 직접전달로 요청 처리 성능이 좋으나, 연결요청 실패시 재시도 x (readneiss probe 헬스체크 결합하여 사용)
- (3) IPVS 모드
    - 커널 공간 동작 및 데이터 구조 해시 테이블로 저장
    - 리눅스 커널에 있는 L4 로드 밸런싱 기술
    - 로드 밸런싱 알고리즘 여러가지 제공 (라운드 로빈… 등)
